In [51]:
import numpy as np
import pandas as pd
from datetime import datetime
import pysolar.solar as solar
import sqlite3
from metar import Metar
import metar_tools

efhk_lat = 60.3183
efhk_lon = 24.9497

ilzq_lat = 60.6277
ilzq_lon = 25.1934

In [52]:
cols = ['ttime', 'content']
efhk_metar = pd.read_csv('sample-data/EFHK_2021_METAR.csv', sep=';', usecols=cols)
efhk_metar['time'] = pd.to_datetime(efhk_metar['ttime'])
efhk_metar.set_index('time', inplace=True)

efhk_metar.index

DatetimeIndex(['2021-01-01 00:20:00', '2021-01-01 00:50:00',
               '2021-01-01 01:20:00', '2021-01-01 01:50:00',
               '2021-01-01 02:20:00', '2021-01-01 02:50:00',
               '2021-01-01 03:20:00', '2021-01-01 03:50:00',
               '2021-01-01 04:20:00', '2021-01-01 04:50:00',
               ...
               '2021-12-03 05:50:00', '2021-12-03 06:20:00',
               '2021-12-03 06:50:00', '2021-12-03 07:20:00',
               '2021-12-03 07:50:00', '2021-12-03 08:20:00',
               '2021-12-03 08:50:00', '2021-12-03 09:20:00',
               '2021-12-03 09:50:00', '2021-12-03 10:20:00'],
              dtype='datetime64[ns]', name='time', length=16140, freq=None)

In [53]:
ilzq_metar = pd.read_csv('data/ILZQ_2021_METAR.dat', sep=';', usecols=cols)
ilzq_metar['time'] = pd.to_datetime(ilzq_metar['ttime'])
ilzq_metar.set_index('time', inplace=True)
ilzq_metar

,ttime,content
time,,
2021-01-01 00:00:00,2021-01-01 00:00:00,ILZQ 010000Z AUTO 23003KT 2300 BR ////// 00/00...
2021-01-01 00:10:00,2021-01-01 00:10:00,ILZQ 010010Z AUTO 23003KT 2100 BR ////// M00/M...
2021-01-01 00:20:00,2021-01-01 00:20:00,ILZQ 010020Z AUTO 23003KT 1800 BR ////// M00/M...
2021-01-01 00:40:00,2021-01-01 00:40:00,ILZQ 010040Z AUTO 22003KT 1700 BR ////// M00/M...
2021-01-01 00:50:00,2021-01-01 00:50:00,ILZQ 010050Z AUTO 23003KT 1800 BR ////// M00/M...
...,...,...
2021-12-03 16:40:00,2021-12-03 16:40:00,ILZQ 031640Z AUTO 35005KT 4200 -SN OVC020 M08/...
2021-12-03 16:50:00,2021-12-03 16:50:00,ILZQ 031650Z AUTO 35004KT 4700 -SN OVC021 M08/...
2021-12-03 17:00:00,2021-12-03 17:00:00,ILZQ 031700Z AUTO 36004KT 5000 -SN OVC030 M08/...


In [56]:
metar = Metar.Metar(efhk_metar.iloc[16, 1])
print(metar.string())

station: EFHK
type: routine report, cycle 8 (automatic report)
time: Sat Jan  1 08:20:00 2022
temperature: 0.0 C
dew point: 0.0 C
wind: ESE at 2 knots
visibility: 800 meters
visual range: on runway 04L, greater than 2000 meters; on runway 15, greater than 2000 meters; on runway 22L, 2000 meters; on runway 22R, greater than 2000 meters
pressure: 1011.0 mb
weather: fog
sky: scattered clouds at 400 feet
     broken clouds at 1300 feet
METAR: EFHK 010820Z 11002KT 0800 R04L/P2000N R15/P2000N R22L/2000N R22R/P2000N FG SCT004 BKN013 00/00 Q1011 BECMG 2000 BR=


In [57]:
#metar.runway[0][1].value()
metar.runway[0][1].value()

2000.0

In [58]:
metar.sky

[('SCT', <metar.Datatypes.distance at 0x1f0ff38fca0>, None),
 ('BKN', <metar.Datatypes.distance at 0x1f085808070>, None)]

In [60]:
dt = datetime.fromisoformat(efhk_metar.iloc[16, 0] + '+00:00')
print(solar.get_altitude(efhk_lat, efhk_lon, dt))

3.2038041011444016


In [61]:
temp = metar.temp.value()
temp

0.0

In [62]:
vis = int(metar.vis.value())
vis

800

In [63]:
metar_tools.get_rvr(metar)

2000

In [64]:
metar_tools.get_ceil(metar)

1300

In [65]:
metar_tools.get_base(metar, metar_tools.get_ceil(metar))

400

In [67]:
row = 0 #42
ilzq_obs = Metar.Metar(ilzq_metar.iloc[row, 1])
print(ilzq_obs.string())

station: ILZQ
type: routine report (automatic report)
time: Sat Jan  1 00:00:00 2022
temperature: 0.0 C
dew point: 0.0 C
wind: SW at 3 knots
visibility: 2300 meters
pressure: 1009.0 mb
weather: mist; ,  and 
METAR: ILZQ 010000Z AUTO 23003KT 2300 BR ////// 00/00 Q1009=


In [68]:
print(ilzq_obs.sky)

[]


In [69]:
metar_tools.get_ceil(ilzq_obs)

9999

In [70]:
metar_tools.get_base(ilzq_obs, metar_tools.get_ceil(ilzq_obs))

9999

In [71]:
metar_tools.is_night('2021-01-01 06:10:00', efhk_lat, efhk_lon)

True

In [72]:
t = datetime.strptime('2021-01-01 06:10:00', '%Y-%m-%d %H:%M:%S')

In [73]:
def extract_metar(data: np.ndarray, lat: float, lon: float):
    time_str = data[0]
    metar_str = data[1]
    obs = Metar.Metar(metar_str, strict=False)
    temp = metar_tools.get_temp(obs)
    vis = metar_tools.get_vis(obs)
    rvr = metar_tools.get_rvr(obs)
    ceil = metar_tools.get_ceil(obs)
    base = metar_tools.get_base(metar, ceil)
    night = metar_tools.is_night(time_str, efhk_lat, efhk_lon)
    return (time_str, temp, vis, rvr, ceil, base, night)

In [74]:
a = [extract_metar(d, lat=efhk_lat, lon=efhk_lon) for d in zip(efhk_metar.ttime, efhk_metar.content)]

C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body '400S' while processing 'EFHK 010250Z 32002KT 0900 400S R04L/1300D R15/1200U R22L/0650D R22R/1600U FG BKN002 M12/M13 Q0994 BECMG 2000 BR='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'BRFG -SHRA BKN002 FEW020CB' while processing 'EFHK 190350Z 36005KT 0800 R04L/P2000N R15/1500D R22L/1700D R22R/1800D -RA FG OVC002 12/12 Q0999 TEMPO 1200 BRFG -SHRA BKN002 FEW020CB='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups in body 'NSW' while processing 'EFHK 261120Z 31004KT 230V350 9999 TS FEW025 FEW030CB 23/17 Q1011 BECMG FEW030 NSW='
  warnings.warn(message, RuntimeWarning)
C:\Users\makes\miniconda3\envs\hems\lib\site-packages\metar\Metar.py:502: RuntimeWarning: Unparsed groups

In [75]:
b = [extract_metar(d, lat=ilzq_lat, lon=ilzq_lon) for d in zip(ilzq_metar.ttime, ilzq_metar.content)]

In [83]:
pd.DataFrame(b, columns=['temp', 'vis', 'rvr', 'ceil', 'base', 'night'])

,temp,vis,rvr,ceil,base,night
0,0,2300,9999,9999,400,True
1,0,2100,9999,9999,400,True
2,0,1800,9999,9999,400,True
3,0,1700,9999,9999,400,True
4,0,1800,9999,9999,400,True
...,...,...,...,...,...,...
45263,-8,4200,9999,2000,400,True
45264,-8,4700,9999,2100,400,True
45265,-8,5000,9999,3000,400,True
45266,-8,5000,9999,2900,400,True


In [81]:
efhk_metar10 = efhk_metar.asfreq('10min', method='ffill')

In [82]:
efhk_metar10.iloc[:10,:]

,ttime,content
time,,
2021-01-01 00:20:00,2021-01-01 00:20:00,EFHK 010020Z 21002KT 0700 R04L/P2000N R15/1400...
2021-01-01 00:30:00,2021-01-01 00:20:00,EFHK 010020Z 21002KT 0700 R04L/P2000N R15/1400...
2021-01-01 00:40:00,2021-01-01 00:20:00,EFHK 010020Z 21002KT 0700 R04L/P2000N R15/1400...
2021-01-01 00:50:00,2021-01-01 00:50:00,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...
2021-01-01 01:00:00,2021-01-01 00:50:00,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...
2021-01-01 01:10:00,2021-01-01 00:50:00,EFHK 010050Z 20002KT 1400 R04L/P2000N R15/P200...
2021-01-01 01:20:00,2021-01-01 01:20:00,EFHK 010120Z 00000KT 2500 BR BKN003 M00/M00 Q1...
2021-01-01 01:30:00,2021-01-01 01:20:00,EFHK 010120Z 00000KT 2500 BR BKN003 M00/M00 Q1...
2021-01-01 01:40:00,2021-01-01 01:20:00,EFHK 010120Z 00000KT 2500 BR BKN003 M00/M00 Q1...
